In [1]:
import requests
from requests.auth import HTTPBasicAuth
from ipython_secrets import get_secret, delete_secret

In [11]:
lrs_url = "https://latte.proj.lip6.fr/lrs"
#
whoami_endpoint = f"{lrs_url}/whoami"
statements_endpoint =  f"{lrs_url}/xAPI/statements"
print (f"LRS base URL:            {lrs_url}")
print (f"  `whoami` endpoint:     {whoami_endpoint}")
print (f"  `statements` endpoint: {statements_endpoint}")

LRS base URL:            https://latte.proj.lip6.fr/lrs
  `whoami` endpoint:     https://latte.proj.lip6.fr/lrs/whoami
  `statements` endpoint: https://latte.proj.lip6.fr/lrs/xAPI/statements


In [16]:
print(f"Authenticating to {lrs_url}...")
# Log-in as the user who sent the statements you want to retrieve.
# That is, the 'authority' behind the statements.
user = get_secret("LRS_USER", prompt="User: ")
password = get_secret("LRS_PASSWORD", prompt="Password: ")
auth = HTTPBasicAuth(user, password)
try:
    response = requests.get(whoami_endpoint, auth=auth)
    print(response)
    if response.status_code != 200:
        if response.status_code == 401:
            raise ValueError('Invalid credentials') 
        elif response.status_code == 404:
            raise ValueError(f"LRS endpoint {whoami_endpoint} not found")
        else:
            raise ValueError('Unknown error')
except Exception:
    delete_secret("LRS_USER")
    delete_secret("LRS_PASSWORD")
o = response.json()
print(o)


Authenticating to https://latte.proj.lip6.fr/lrs...
<Response [200]>
{'agent': {'mbox': 'mailto:pierre.thiel@lip6.fr', 'objectType': 'Agent', 'name': 'lrs-sandbox-test'}, 'scopes': ['all']}


In [13]:
# Retrieve statements 'belonging' to the authenticated user (within limit of LRS)
response = requests.get(statements_endpoint, auth=auth, params={'mine': True})
res_dict = response.json()
statements = res_dict['statements']
print(f"Retrieved {len(statements)} statements")

Retrieved 100 statements


In [18]:
# Retrieve statements 'belonging' to the authenticated user, up to a given limit
statements = []
limit = 1000
response = requests.get(statements_endpoint, auth=auth, params={'mine': True, 'limit': limit})
res_dict = response.json()
statements += res_dict['statements']
print(f"Got {len(statements)} results")
while len(statements) < limit and 'more' in res_dict.keys():
    next_result_endpoint = f"{lrs_url}{res_dict['more']}"
    print(f"Getting more results at {next_result_endpoint}")
    response = requests.get(next_result_endpoint , auth=auth)
    res_dict = response.json()
    print(len(res_dict['statements']))
    statements += res_dict['statements']
print(f"Retrieved {len(statements)} statements")

Got 100 results
Getting more results at https://latte.proj.lip6.fr/lrs/xAPI/statements?mine=True&limit=1000&pit_id=8_LoAwEKc3RhdGVtZW50cxZvYjdiOHdialEwbThxSWNRYkhLanRBABZWdVFuRDBtTVMzV1JHbzFfN0Z0dnJnAAAAAAAAACxEFjVsMng3TEZzUklpWUw0RE9XUUdhMEEAARZvYjdiOHdialEwbThxSWNRYkhLanRBAAA%3D&search_after=1542200658650%7C8
0
Retrieved 100 statements
